# HAND and Flood Emergency Response

This Jupyter notebook illustrates the HAND workflow and its use in example flood emergency scenarios. The study area is Onion Creek (HUC10 code 1209020504).

This is also a demonstration of conducting geospatial anlysis with opensource toolkits (gdal) using an online Jupyter interface.

Environment required: CyberGIS-Jupyter for Water

![](https://camo.githubusercontent.com/b4a3f72e68690da6ee5726fe2f2d8711c9d314ce/68747470733a2f2f7765622e636f7272616c2e746163632e7574657861732e6564752f6e666965646174612f646f63732f68616e642d776f726b666c6f772e706e67)

- We use CyberGIS' accelerated TauDEM version for d8 and d$\infty$ flow direction calculation.
    >More about [TauDEM](http://hydrology.usu.edu/taudem/taudem5/index.html)


- The USGS 3DEP Elevation dataset (a.k.a. National Elevation Dataset) is deployed on ROGER as a VRT raster.

- The NHDPlus and associated water boundary dataset (WBD) are in Esri FileGDB format.

# Notebook Outline:
- [Data preparation and pre-processing](#preparation)
- [Workflow for computing HAND](#Workflow)
    - [Find inlets](#Workflow)
    - [Pitremove](#Pitremove)
    - [Compute flow direction](#Direction)
    - [Calculate HAND](#Hand)
- [Application](#Application)  

<a id='preparation'></a>
# 1. Data preparation and pre-processing

In [1]:
# Program file path
import os
data_dir = '/home/jovyan/shared_data/data/onion-hand'
    
# Format and performance parameters
np=4 #os.cpu_count()
dsepsg="EPSG:4269"
bufferdist='0.2' 

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Fetch the hands-on example data, from [*GIS in Water Resources*](http://hydrology.usu.edu/dtarb/giswr/2016/) (USU CEE6440, UT Austin CE 394K.3)

In [2]:
!cp -r {data_dir} data

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
The study area is Onion Creek in Texas. This watershed is a HUC10 unit with code 1209020504.
<!--In this demo notebook, we directly use flowline shp file for the input DEM.<br>
Extracting flowlines from NHDPlus is left as an exercise for you to get familiar with GDAL OGR utility. <br>-->
 <!--You can get the WBD for this unit and conduct a spatial query on NHDPlus to get the flowlines.--></span>

In [3]:
!ogrinfo data/OnionHand.gdb

INFO: Open of `data/OnionHand.gdb'
      using driver `OpenFileGDB' successful.
1: StreamGage (3D Measured Point)
2: NHDFlowline (3D Measured Multi Line String)
3: Catchment (Multi Polygon)
4: Watershed (Multi Polygon)
5: AddressPt (Point)


In [4]:
!gdalinfo data/Onion3.tif

Driver: GTiff/GeoTIFF
Files: data/Onion3.tif
Size is 7864, 2761
Coordinate System is:
GEOGCRS["NAD83",
    DATUM["North American Datum 1983",
        ELLIPSOID["GRS 1980",6378137,298.257222101,
            LENGTHUNIT["metre",1]],
        ID["EPSG",6269]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["Degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["longitude",east,
            ORDER[1],
            ANGLEUNIT["Degree",0.0174532925199433]],
        AXIS["latitude",north,
            ORDER[2],
            ANGLEUNIT["Degree",0.0174532925199433]]]
Data axis to CRS axis mapping: 1,2
Origin = (-98.307685185190977,30.270277778898251)
Pixel Size = (0.000092592593000,-0.000092592593000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  ( -98.3076852,  30.2702778) ( 98d18'27.67"W, 30d16'13.00"N)
Lower Left  ( -98.3076852,  30.0146296) ( 98d18'27.67"W, 30d 0'52.67"N)
Upper Right ( -97.5795370,  30.2702778) 

>More about [ogrinfo](http://www.gdal.org/ogrinfo.html) and [gdalinfo](http://www.gdal.org/gdalinfo.html)

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
To visualize a GeoTIFF, we need to:
 - Render color from values.
 - Make tiles (pyramids) from the colored image.

In [5]:
# Colorize using hillshade
!gdaldem hillshade data/Onion3.tif Onion3-hillshade.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [6]:
# Make tiles with gdal2tiles (this is needed for visualization)
!gdal2tiles.py -t "Onion Creek DEM" -e -z 9-14 -a 0,0,0 -s epsg:4326\
    -r bilinear Onion3-hillshade.tif Onion3-hillshade 

Warning 1: VRTDataset>, band 1: NODATA_VALUES metadata item doesn't have the same number of values as the number of bands.  Ignoring it for mask.
Generating Base Tiles:
Warning 1: /tmp/tmp672amor7/b857b5f6-5906-4864-8721-bf01a0fedad3.vrt, band 1: NODATA_VALUES metadata item doesn't have the same number of values as the number of bands.  Ignoring it for mask.
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


>More about [gdaldem](http://www.gdal.org/gdaldem.html) and [gdal2tiles.py](http://www.gdal.org/gdal2tiles.html)

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
To visualize a Shapefile, we only need to convert it to GeoJson

In [7]:
# Extract flowline.shp from GeoDatabase (which will be used later)
!ogr2ogr -f "ESRI Shapefile" flowline.shp data/OnionHand.gdb NHDFlowline 2>/dev/null

In [8]:
# Convert the Shapefile to GeoJson for visualization
!ogr2ogr -f geojson flowline.json flowline.shp

Warning 1: The output driver does not natively support Date type for field FDATE. Misconversion can happen. -mapFieldType can be used to control field type conversion.


>More about [ogr2ogr](http://www.gdal.org/ogr2ogr.html)

In [9]:
from cybergis import Floret
Floret('Onion Creek DEM', 'input')\
.addTMSLayer('DEM','Onion3-hillshade')\
.addGeoJson('Flowline','flowline.json').display()

<a id='Workflow'></a>
# 2. Workflow for computing HAND


## 2.1 Find inlets
<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Use the dangle operation to find channel heads (inlets) of streams. They will guide the DEM-based stream calculation.</span>

In [10]:
!find_inlets_mr -flow flowline.shp -dangle flow-inlets0.shp
!ogr2ogr -t_srs "EPSG:4269" flow-inlets.shp flow-inlets0.shp

In [11]:
!ogr2ogr -f geojson flow-inlets.json flow-inlets.shp

In [12]:
from cybergis import Floret
Floret('Flow Inlets', 'inlet').addTMSLayer('DEM','Onion3-hillshade')\
.addGeoJson('Flowline', 'flowline.json')\
.addGeoJson('Inlets','flow-inlets.json')\
.display()

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Rasterize inlet points to the same spatial extent as the input DEM. The output is the weight grid.</span>

In [13]:
ls data/getRasterInfo.py

data/getRasterInfo.py*


In [14]:
# Getting format info of the reference tif
import sys
names='fsizeDEM colsDEM rowsDEM nodataDEM xmin ymin xmax ymax cellsize_resx cellsize_resy'.split()
values=!{sys.executable} data/getRasterInfo.py data/Onion3.tif
print(zip(names, values))
for name,value in dict(zip(names,values)).items():
   para='%s="%s"'%(name,value)
   print (para)
   exec(para)

fsizeDEM="84.6MB"
colsDEM="7864"
rowsDEM="2761"
nodataDEM="-1.0"
xmin="-98.30768518519098"
ymin="30.01462962962525"
xmax="-97.57953703383897"
ymax="30.27027777889825"
cellsize_resx="0.000092592593000"
cellsize_resy="0.000092592593000"


In [15]:
!gdal_rasterize -ot Int16 -of GTiff -burn 1 -tr {cellsize_resx} {cellsize_resy} -te {xmin} {ymin} {xmax} {ymax} flow-inlets.shp weights.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


>More about [gdal_rasterize](http://www.gdal.org/gdal_rasterize.html)

<a id='Pitremove'></a>
## 2.2 Pitremove

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Taudem pitremove: hydro-condition the DEM

In [16]:
!mpirun -np {np} pitremove -z data/Onion3.tif -fel fel.tif

PitRemove version 5.3.9
Input file data/Onion3.tif has geographic coordinate system.
Nodata value input to create partition from file: -1.000000
Nodata value recast to float used in partition raster: -1.000000
This run may take on the order of 1 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
Processes: 4
Header read time: 0.325979
Data read time: 0.800494
Compute time: 15.243871
Write time: 3.157866
Total time: 19.528210


<a id='Direction'></a>
## 2.3 Compute flow direction

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Taudem d$\infty$: calc flow routing info that is used later to find the nearest stream from each cell</span>

In [17]:
!mpirun -np {np} dinfflowdir -fel fel.tif -ang ang.tif -slp slp.tif 

DinfFlowDir version 5.3.9
Input file fel.tif has geographic coordinate system.
Nodata value input to create partition from file: -300000000549775575777803994281145270272.000000
Nodata value recast to float used in partition raster: -300000000549775575777803994281145270272.000000
This run may take on the order of 5 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
All slopes evaluated. 405327 flats to resolve.
Draining flats towards lower adjacent terrain
............................................................................................................................................................................................................................................................................................................................

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">TauDEM d8: calc d8 flow directions for the computing of a stream grid weighted by the weight grid

In [18]:
!mpirun -np {np} d8flowdir -fel fel.tif -p p.tif -sd8 sd8.tif

D8FlowDir version 5.3.9
Input file fel.tif has geographic coordinate system.
Nodata value input to create partition from file: -300000000549775575777803994281145270272.000000
Nodata value recast to float used in partition raster: -300000000549775575777803994281145270272.000000
This run may take on the order of 5 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
All slopes evaluated. 405327 flats to resolve.
Draining flats towards lower adjacent terrain
..............................................................................................................................................................................................................................................................................................................................

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">TauDEM aread8: calc contributing area with the weight grid. i.e., only the contributing area starting from the channel heads denoted in the weight grid

In [19]:
!mpirun -np {np} aread8 -p p.tif -ad8 ssa.tif -wg weights.tif -nc 

AreaD8 version 5.3.9
Input file p.tif has geographic coordinate system.
Nodata value input to create partition from file: -32768.000000
Nodata value recast to int16_t used in partition raster: -32768
This run may take on the order of 1 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
Input file weights.tif has geographic coordinate system.
Nodata value input to create partition from file: -10000000000.000000
Nodata value recast to float used in partition raster: -10000000000.000000
Number of Processes: 4
Read time: 0.961618
Compute time: 8.446016
Write time: 0.565020
Total time: 9.972653


<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">From the contributing area, extract streams areas (cellvalue = 1)

In [20]:
!mpirun -np {np} threshold -ssa ssa.tif -src src.tif -thresh 1

Threshold version 5.3.9
Input file ssa.tif has geographic coordinate system.
Nodata value input to create partition from file: -1.000000
Nodata value recast to float used in partition raster: -1.000000
This run may take on the order of 1 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
Compute time: 0.751273


In [21]:
!gdaldem color-relief src.tif data/src.clr src-color.tif -of GTiff -alpha -nearest_color_entry
!gdal2tiles.py -e -z 9-14 -a 0,0,0 -s epsg:4326 -r bilinear src-color.tif src-color 

0...10...20...30...40...50...60...70...80...90...100 - done.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [22]:
from cybergis import Floret
Floret('DEM-Derived streamline','src')\
.addTMSLayer('Derived Streamline', 'src-color')\
.addGeoJson('Flowline','flowline.json').display()

<a id='Hand'></a>
## 2.4 Calculate HAND

<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">With the d$\infty$ flowing routing and a stream grid that we know water flows on, calc HAND. We use vertical distance here; but horizontal distance or the combination can be used.

In [23]:
!mpirun -np {np} dinfdistdown -fel fel.tif -ang ang.tif -src src.tif -dd hand.tif -m ave v

DinfDistDown -v version 5.3.9
Input file ang.tif has geographic coordinate system.
Nodata value input to create partition from file: -340282346638528859811704183484516925440.000000
Nodata value recast to float used in partition raster: -340282346638528859811704183484516925440.000000
This run may take on the order of 1 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.
Input file fel.tif has geographic coordinate system.
Nodata value input to create partition from file: -300000000549775575777803994281145270272.000000
Nodata value recast to float used in partition raster: -300000000549775575777803994281145270272.000000
Input file src.tif has geographic coordinate system.
Nodata value input to create partition from file: -32768.000000
Nodata value recast to int16_t use

In [24]:
# Colorization and tiling
!rm -rf ang-color
!gdaldem color-relief ang.tif data/ang.clr ang-color.tif -of GTiff -alpha
!gdal2tiles.py -e -z 9-14 -a 0,0,0 -s epsg:4326 -r bilinear ang-color.tif ang-color 

0...10...20...30...40...50...60...70...80...90...100 - done.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [25]:
from cybergis import Floret
Floret('Dinf Distancedown','ang')\
.addTMSLayer('Dinf', 'ang-color')\
.addGeoJson('Flowline','flowline.json').display()

In [26]:
# Colorization and tiling
!gdaldem color-relief hand.tif data/HAND-blues.clr hand-color.tif -of GTiff -alpha
!gdal2tiles.py -e -z 9-14 -a 0,0,0 -s epsg:4326 -r bilinear hand-color.tif hand-color 

0...10...20...30...40...50...60...70...80...90...100 - done.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100


In [27]:
from cybergis import Floret
Floret('Onion Creek HAND', 'onion-hand')\
.addTMSLayer('DEM', 'Onion3-hillshade')\
.addTMSLayer('HAND', 'hand-color')\
.addGeoJson('Flowline', 'flowline.json')\
.display()

<a id='Application'></a>
# 3. Application
<img src="http://cybergis.cigi.uiuc.edu/images/v5.svg" width="15" style="float:left"/>&nbsp;<span style="font-family:Times; font-size:1.2em">
Whose homes will be flooded if a 5m flood take place?

In [28]:
# Filtering the HAND tif with threshold value
!gdal_calc.py -A hand.tif --outfile=depth.tif --calc="A<=5"

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 9.. 

61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 61.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 62.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 63.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 64.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 65.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 

In [29]:
# Convert the result into polygons
!gdal_polygonize.py depth.tif depth.shp # ed Becky

Creating output depth.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.


>More about [gdal_calc.py](http://www.gdal.org/gdal_calc.py.html) and [gdal_polygonize.py](http://www.gdal.org/gdal_polygonize.html)

In [30]:
!ogrinfo -al -so depth.shp

INFO: Open of `depth.shp'
      using driver `ESRI Shapefile' successful.

Layer name: depth
Metadata:
  DBF_DATE_LAST_UPDATE=2020-07-13
Geometry: Polygon
Feature Count: 700
Extent: (-98.297685, 30.024259) - (-97.579630, 30.270185)
Layer SRS WKT:
GEOGCRS["NAD83",
    DATUM["North American Datum 1983",
        ELLIPSOID["GRS 1980",6378137,298.257222101,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["latitude",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["longitude",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4269]]
Data axis to CRS axis mapping: 2,1
DN: Integer (9.0)


In [31]:
# Filter the target areas from background
!ogr2ogr filtered_depth.shp depth.shp -where "DN=1"

In [32]:
# Convert the result to a GeoJson
!ogr2ogr -f geojson filtered_depth.json filtered_depth.shp

In [33]:
# The catchment of interest and the addressPoints in that catchment
!ogr2ogr -f geojson catchment.json data/catchment.shp
!ogr2ogr -f geojson addressPoints.json data/addressPoints.shp

In [34]:
from cybergis import Floret
m = Floret('Catchment addresses and 5m-flood', '5m')
m.addTMSLayer('HAND', 'hand-color')
m.addGeoJson('Vunerable Areas', 'filtered_depth.json')
m.addGeoJson('Catchment', 'catchment.json')
m.addGeoJson('Address Points', 'addressPoints.json')
m.display()

In [35]:
from cybergis import Editor
area = Editor('filtered_depth.shp')
addr = Editor('data/addressPoints.shp')
result=[]
for i in range(addr.lens):
    if -1 != area.index_of_first_feature_contains_point(*addr.shape(i).points[0]):
        result.append(i)
ans=addr.subshp(result)
ans.save('vunerableAddr')
!ogr2ogr -f geojson vunerableAddr.json vunerableAddr.shp

In [36]:
from cybergis import Floret
m = Floret('Flooded addresses', 'flooded')
m.addTMSLayer('HAND', 'hand-color')
m.addGeoJson('Vunerable Areas', 'filtered_depth.json')
m.addGeoJson('Catchment', 'catchment.json')
m.addGeoJson('Vunerable Addresses', 'vunerableAddr.json')
m.display()

# Congratulations on reaching the end! 